In [1]:
import ee
import json
from pathlib import Path
import httplib2
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as disp
from osgeo import gdal

Collecting MOD09A1 Imagery

In [2]:
import ee
import json
from pathlib import Path
import httplib2
import time

ee.Authenticate()

geojson_path = list(Path(Path.cwd()/"Data"/"GeoJson").resolve().iterdir())[0]
geojson_path

with open(geojson_path) as f:
    data = json.load(f)
#"Kenya","Malawi"
coords = {}
countries = ["Kenya","Malawi","Ethiopia"]
for j in countries:
    for i in data['features']:
        if i['properties']['name'] == j:
            coords[j] = i['geometry']['coordinates'][0]
            break


def appendBands(now,running):
    running = ee.Image(running)
    now = now.select([0,1,2,3,4,5,6])
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(running,None),now,running.addBands(ee.Image(now)))
    return accum


def exportImage(img,desc,region):
    task = ee.batch.Export.image.toDrive(image=img,folder = f"{desc}_Surface",description = desc,region=region,scale=500,crs = 'EPSG:4326')
    task.start()
    while task.status()['state'] == "RUNNING":
        print("Running...")
        time.sleep(10)
    print(f"Done,{task.status()}")


_http_transport = httplib2.Http(disable_ssl_certificate_validation=True)
ee.Initialize(http_transport=_http_transport)

for country in countries:
    aoi = ee.Geometry.Polygon(coords[country])
    imgcoll = ee.ImageCollection('MODIS/MOD09A1')\
                                .filterBounds(aoi)\
                                .filterDate(ee.Date('2009-01-01'),ee.Date('2022-01-01'))
    print(f"Number of images : {imgcoll.size().getInfo()}")
    img = imgcoll.iterate(appendBands)
    img = ee.Image(img)
    try:
        exportImage(img,country,aoi)
    except:
        print(f'Failed:{country}')


Successfully saved authorization token.
Number of images : 378
Done,{'state': 'READY', 'description': 'Kenya', 'creation_timestamp_ms': 1649605207493, 'update_timestamp_ms': 1649605207493, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '7EN3M256A34GAOBZ3Y26I7UC', 'name': 'projects/earthengine-legacy/operations/7EN3M256A34GAOBZ3Y26I7UC'}
Number of images : 378
Done,{'state': 'READY', 'description': 'Malawi', 'creation_timestamp_ms': 1649605210174, 'update_timestamp_ms': 1649605210174, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'NIUNNXWW4TXOXACXWI2VA35Z', 'name': 'projects/earthengine-legacy/operations/NIUNNXWW4TXOXACXWI2VA35Z'}
Number of images : 378
Done,{'state': 'READY', 'description': 'Ethiopia', 'creation_timestamp_ms': 1649605212784, 'update_timestamp_ms': 1649605212784, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'PEDO5AD52J36I2J3UZAUSDF2', 'name': 'projects/earthengine-legacy/operations/PEDO5AD52J36I2J3UZAUSDF2'}


Collection MCD12Q1 Land Cover Imagery

Each landcover image has one useful Band LC_Type1.

We append the bands of each image in the image collection to get one big image. 

You get one image per year for landcover

In [3]:
# ee.Authenticate()
# _http_transport = httplib2.Http(disable_ssl_certificate_validation=True)
# ee.Initialize(http_transport=_http_transport)

geojson_path = list(Path(Path.cwd()/"Data"/"GeoJson").resolve().iterdir())[0]
geojson_path

with open(geojson_path) as f:
    data = json.load(f)


def appendBands_landCover(now,running):
    running = ee.Image(running)
    #Select the Annual IGBP Classification
    now = now.select([0])
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(running,None),now,running.addBands(ee.Image(now)))
    return accum

def exportImage(img,desc,folder,region):
    task = ee.batch.Export.image.toDrive(image = img,description=desc,folder = f"{folder}_LandCover",region = region,scale=500,crs = 'EPSG:4326')
    task.start()
    while task.status()['state'] == "RUNNING":
        print("Running...")
        time.sleep(10)
    print(f"Done,{task.status()}")

coords = {}
countries = ["Kenya","Ethiopia","Malawi"]
for j in countries:
    for i in data['features']:
        if i['properties']['name'] == j:
            coords[j] = i['geometry']['coordinates'][0]
            break

for country in countries:
    aoi_landcover = ee.Geometry.Polygon(coords[country])
    landcovercol = ee.ImageCollection('MODIS/006/MCD12Q1')\
                                    .filterBounds(aoi_landcover)\
                                    .filterDate(ee.Date('2009-01-01'),ee.Date('2022-01-01'))   
    #append the bands of each image in the collection into one band
    landcovercol = landcovercol.iterate(appendBands_landCover)
    #make the appended into an image again
    landcovercol = ee.Image(landcovercol)
    try:
       exportImage(landcovercol,desc = country,folder = country,region = aoi_landcover)
    except:
        print(f"Failed:{country}")        

Done,{'state': 'READY', 'description': 'Kenya', 'creation_timestamp_ms': 1649605214835, 'update_timestamp_ms': 1649605214835, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'T4INE2PCB4ODK7HKCVFSKIOJ', 'name': 'projects/earthengine-legacy/operations/T4INE2PCB4ODK7HKCVFSKIOJ'}
Done,{'state': 'READY', 'description': 'Ethiopia', 'creation_timestamp_ms': 1649605217101, 'update_timestamp_ms': 1649605217101, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '2T2UHENAN5W7OCKUMTUDQNLS', 'name': 'projects/earthengine-legacy/operations/2T2UHENAN5W7OCKUMTUDQNLS'}
Done,{'state': 'READY', 'description': 'Malawi', 'creation_timestamp_ms': 1649605219415, 'update_timestamp_ms': 1649605219415, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'OD2YLT4QDJSRLVJNYGI7CVEQ', 'name': 'projects/earthengine-legacy/operations/OD2YLT4QDJSRLVJNYGI7CVEQ'}


In [4]:
# ee.Authenticate()
# _http_transport = httplib2.Http(disable_ssl_certificate_validation=True)
# ee.Initialize(http_transport=_http_transport)


# def appendBands_landCover(now,running):
#     running = ee.Image(running)
#     #Select the Annual IGBP Classification
#     now = now.select([0])
#     accum = ee.Algorithms.If(ee.Algorithms.IsEqual(running,None),now,running.addBands(ee.Image(now)))
#     return accum


# # imgcoll = ee.ImageCollection('MODIS/MOD09A1') \
# #     .filterBounds(ee.Geometry.Rectangle(-106.5, 50,-64, 23))\
# #     .filterDate('2001-12-31','2015-12-31')
# imgcoll = ee.ImageCollection('MODIS/006/MCD12Q1') \
#     .filterBounds(ee.Geometry.Rectangle(-106.5, 50,-64, 23))\
#     .filterDate('2001-12-31','2015-12-31')
# # img=imgcoll.iterate(appendBands_landCover)

In [5]:
# ee.Authenticate()
# _http_transport = httplib2.Http(disable_ssl_certificate_validation=True)
# ee.Initialize(http_transport=_http_transport)

geojson_path = list(Path(Path.cwd()/"Data"/"GeoJson").resolve().iterdir())[0]
geojson_path

with open(geojson_path) as f:
    data = json.load(f)

#Bands being used : 'LST_Day_1km', 'LST_Night_1km'
def appendBands_landCover(now,running):
    running = ee.Image(running)
    #Select the Annual IGBP Classification
    now = now.select([0,4])
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(running,None),now,running.addBands(ee.Image(now)))
    return accum

def exportImage(img,desc,folder,region):
    task = ee.batch.Export.image.toDrive(image = img,description=desc,folder = f"{folder}_Temperature",region = region,scale=500,crs = 'EPSG:4326')
    task.start()
    while task.status()['state'] == "RUNNING":
        print("Running...")
        time.sleep(10)
    print(f"Done,{task.status()}")

coords = {}
countries = ["Kenya","Ethiopia","Malawi"]
for j in countries:
    for i in data['features']:
        if i['properties']['name'] == j:
            coords[j] = i['geometry']['coordinates'][0]
            break

for country in countries:
    aoi_temp = ee.Geometry.Polygon(coords[country])
    tempcol = ee.ImageCollection('MODIS/MYD11A2')\
                                    .filterBounds(aoi_temp)\
                                    .filterDate(ee.Date('2009-01-01'),ee.Date('2022-01-01'))   
    #append the bands of each image in the collection into one band
    tempcol = tempcol.iterate(appendBands_landCover)
    #make the appended into an image again
    tempcol = ee.Image(tempcol)
    try:
       exportImage(tempcol,desc = country,folder = country,region = aoi_temp)
    except:
        print(f"Failed:{country}")        

Done,{'state': 'READY', 'description': 'Kenya', 'creation_timestamp_ms': 1649605222172, 'update_timestamp_ms': 1649605222172, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'SICWPLNTFNZXZMUZVF567DPI', 'name': 'projects/earthengine-legacy/operations/SICWPLNTFNZXZMUZVF567DPI'}
Done,{'state': 'READY', 'description': 'Ethiopia', 'creation_timestamp_ms': 1649605223985, 'update_timestamp_ms': 1649605223985, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'MLGNDTJCRYW6PKFJUUJNIIZG', 'name': 'projects/earthengine-legacy/operations/MLGNDTJCRYW6PKFJUUJNIIZG'}
Done,{'state': 'READY', 'description': 'Malawi', 'creation_timestamp_ms': 1649605225627, 'update_timestamp_ms': 1649605225627, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'X3BAQ5LNVRSINWRXVRTTOT3T', 'name': 'projects/earthengine-legacy/operations/X3BAQ5LNVRSINWRXVRTTOT3T'}


Required Code ends at this point. All other code is test code

In [ ]:
aoi_landcover = ee.Geometry.Polygon(coords["Kenya"])
tempcol1 = ee.ImageCollection('MODIS/MYD11A2')\
                                    .filterBounds(aoi_landcover)\
                                    .filterDate(ee.Date('2009-01-01'),ee.Date('2022-01-01'))   

In [ ]:
aoi = ee.Geometry.Polygon(coords["Kenya"])
imgcol = ee.ImageCollection('MODIS/MOD09A1')\
                                .filterBounds(aoi)\
                                .filterDate(ee.Date('2009-01-01'),ee.Date('2022-01-01'))

In [ ]:
aoi_landcover = ee.Geometry.Polygon(coords["Kenya"])
landcovercol = ee.ImageCollection('MODIS/006/MCD12Q1')\
                                .filterBounds(aoi_landcover)\
                                .filterDate(ee.Date('2009-01-01'),ee.Date('2022-01-01'))   

In [ ]:
imgcol.first().select([0]).arrayDimensions().getInfo()

{'type': 'Image',
 'bands': [{'id': 'sur_refl_b01',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'dimensions': [86400, 43200],
   'crs': 'SR-ORG:6974',
   'crs_transform': [463.3127165279165,
    0,
    -20015109.353988,
    0,
    -463.3127165274999,
    10007554.676994]}],
 'version': 1505449763721124,
 'id': 'MODIS/MOD09A1/MOD09A1_005_2009_01_01',
 'properties': {'system:time_start': 1230768000000,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 1231459200000,
  'system:asset_size': 23210964357,
  'system:index': 'MOD09A1_005_2009_01_01'}}

In [ ]:
tempcol1.first().select([0]).arrayDimensions().getInfo()

{'type': 'Image',
 'bands': [{'id': 'LST_Day_1km',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'dimensions': [43200, 21600],
   'crs': 'SR-ORG:6974',
   'crs_transform': [926.625433055833,
    0,
    -20015109.353988,
    0,
    -926.6254330549998,
    10007554.676994]}],
 'version': 1505449767505268,
 'id': 'MODIS/MYD11A2/MYD11A2_005_2009_01_01',
 'properties': {'system:time_start': 1230768000000,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 1231459200000,
  'system:asset_size': 1073728585,
  'system:index': 'MYD11A2_005_2009_01_01'}}

In [ ]:
#27 images per year (13 years)
imgcoll.size().getInfo()

378

In [ ]:
#27 images per year(ignore the last one)
tempcol.size().getInfo()

379

In [ ]:
img.bandNames().getInfo()

['LST_Day_1km',
 'QC_Day',
 'Day_view_time',
 'Day_view_angl',
 'LST_Night_1km',
 'QC_Night',
 'Night_view_time',
 'Night_view_angl',
 'Emis_31',
 'Emis_32',
 'Clear_sky_days',
 'Clear_sky_nights']

In [ ]:
img.select([0,4]).bandNames().getInfo()

['LST_Day_1km', 'LST_Night_1km']

In [ ]:
imgcoll = ee.ImageCollection('MODIS/MOD09A1') \
    .filterBounds(ee.Geometry.Rectangle(-106.5, 50,-64, 23))\
    .filterDate('2001-12-31','2015-12-31')

In [ ]:
imgcoll.first().select([0]).arrayDimensions().getInfo()

{'type': 'Image',
 'bands': [{'id': 'sur_refl_b01',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'dimensions': [86400, 43200],
   'crs': 'SR-ORG:6974',
   'crs_transform': [463.3127165279165,
    0,
    -20015109.353988,
    0,
    -463.3127165274999,
    10007554.676994]}],
 'version': 1505449805657819,
 'id': 'MODIS/MOD09A1/MOD09A1_005_2002_01_01',
 'properties': {'system:time_start': 1009843200000,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 1010534400000,
  'system:asset_size': 22803537727,
  'system:index': 'MOD09A1_005_2002_01_01'}}

In [ ]:
lccoll = ee.ImageCollection('MODIS/006/MCD12Q1') \
    .filterBounds(ee.Geometry.Rectangle(-106.5, 50,-64, 23))\
    .filterDate('2001-12-31','2015-12-31')

In [ ]:
lccoll.size().getInfo()

14

In [ ]:
tempcoll = ee.ImageCollection('MODIS/MYD11A2') \
    .filterBounds(ee.Geometry.Rectangle(-106.5, 50,-64, 23))\
    .filterDate('2002-12-31','2015-12-31')

tempcoll = tempcoll.iterate(appendBands_landCover)
tempimg = ee.Image(tempcoll)
exportImage(tempimg,desc = "Temp",folder = "Temp",region = ee.Geometry.Rectangle(-106.5, 50,-64, 23))

Done,{'state': 'READY', 'description': 'Temp', 'creation_timestamp_ms': 1649602910429, 'update_timestamp_ms': 1649602910429, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'BO36CQO555QQYTOQGAZUDPY6', 'name': 'projects/earthengine-legacy/operations/BO36CQO555QQYTOQGAZUDPY6'}


In [ ]:
def appendBands(now,running):
    running = ee.Image(running)
    now = now.select([0,1,2,3,4,5,6])
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(running,None),now,running.addBands(ee.Image(now)))
    return accum


def exportImage(img,desc,region):
    task = ee.batch.Export.image.toDrive(image=img,folder = f"{desc}_Surface",description = desc,region=region)
    task.start()
    while task.status()['state'] == "RUNNING":
        print("Running...")
        time.sleep(10)
    print(f"Done,{task.status()}")

imgcoll = ee.ImageCollection('MODIS/MOD09A1') \
    .filterBounds(ee.Geometry.Rectangle(-106.5, 50,-64, 23))\
    .filterDate('2001-12-31','2015-12-31')

imgcoll = imgcoll.iterate(appendBands_landCover)
img = ee.Image(imgcoll)
exportImage(img,desc = "Temp",region = ee.Geometry.Rectangle(-106.5, 50,-64, 23))

Done,{'state': 'READY', 'description': 'Temp', 'creation_timestamp_ms': 1649603338103, 'update_timestamp_ms': 1649603338103, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '35JY7YXCTG3CBVUMV4UJTVKQ', 'name': 'projects/earthengine-legacy/operations/35JY7YXCTG3CBVUMV4UJTVKQ'}


In [ ]:
tempcoll.first().select([0]).arrayDimensions().getInfo()

{'type': 'Image',
 'bands': [{'id': 'LST_Day_1km',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'dimensions': [43200, 21600],
   'crs': 'SR-ORG:6974',
   'crs_transform': [926.625433055833,
    0,
    -20015109.353988,
    0,
    -926.6254330549998,
    10007554.676994]}],
 'version': 1505449760317145,
 'id': 'MODIS/MYD11A2/MYD11A2_005_2003_01_01',
 'properties': {'system:time_start': 1041379200000,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 1042070400000,
  'system:asset_size': 1073567625,
  'system:index': 'MYD11A2_005_2003_01_01'}}